In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import pandas as pd

load_dotenv()
engine = create_engine(f"postgresql://{os.environ['DB_USERNAME']}:{os.environ['DB_PASSWORD']}@{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_DATABASE']}")

In [3]:
game_table = "game"
df = pd.read_sql_table(table_name=game_table, con=engine)

In [4]:
df.head()

,id,name,platform,rating,genre,num_of_player,img_url
0,1,#DRIVE,Switch,all,"['Racing', 'Arcade', 'Automobile']",No Online Multiplayer,https://static.metacritic.com/images/products/...
1,2,#IDARB,Xbox One,all,"['Action', 'Miscellaneous', 'General', 'Party ...",At Least 1,https://static.metacritic.com/images/products/...
2,3,#KILLALLZOMBIES,Xbox One,13+,"['Action', 'Shooter', ""Shoot-'Em-Up"", 'Top-Down']",No Online Multiplayer,https://static.metacritic.com/images/products/...
3,4,'Splosion Man,Xbox 360,10+,"['Action', 'General', 'Platformer', 'Platforme...",4 Online,https://static.metacritic.com/images/products/...
4,5,.detuned,PlayStation 3,all,"['Action', 'Miscellaneous', 'Rhythm', 'Music',...",1 Player,https://static.metacritic.com/images/products/...


In [5]:
k = []
for i in df['genre']:
    k.append(i[1:-1])
df['genre'] = k

In [6]:
df.isna().sum()

id               0
name             0
platform         0
rating           0
genre            0
num_of_player    0
img_url          0
dtype: int64

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
################## 유저가 플레이한 게임 데이터 입력 받기 (예시) #################
user_games_names = ['playerunknowns-battlegrounds', 'FIFA 20']
#user_games_names = ['Dwarf Fortress']

# 문자열 배열의 요소를 소문자로 변환하고 공백을 '-'로 대체
user_games_names_modified = [name.lower().replace(' ', '-') for name in user_games_names]

In [9]:
user_games_names_modified         

['playerunknowns-battlegrounds', 'fifa-20']

In [10]:
from MetaCriticScraper import MetaCriticScraper

# 빈 데이터 프레임 생성
user_games = pd.DataFrame(columns=['name', 'genre'])

gnere_list = ['pc', 'playStation-4','playStation-5', 'switch', 'xbox-one', 'xbox-series-x'] 
for j in user_games_names_modified:
    for i in gnere_list:
        url = 'https://www.metacritic.com/game/'+ i + '/' + j
        scraper = MetaCriticScraper(url)
        # 데이터 프레임에 행 추가
        user_games = pd.concat([user_games, pd.DataFrame({'name': scraper.game['title'], 'genre': [','.join(scraper.game['genre'])]}, index=[0])], ignore_index=True)
        if scraper.game['url']:
            break

In [11]:
user_games

,name,genre
0,PlayerUnknown's Battlegrounds,"Action,Shooter,First-Person,Tactical"
1,FIFA 20,"Sports,Team,Soccer,Sim"


In [12]:
db_df = df[['name', 'genre']]

In [13]:
db_df.head()

,name,genre
0,#DRIVE,"'Racing', 'Arcade', 'Automobile'"
1,#IDARB,"'Action', 'Miscellaneous', 'General', 'Party /..."
2,#KILLALLZOMBIES,"'Action', 'Shooter', ""Shoot-'Em-Up"", 'Top-Down'"
3,'Splosion Man,"'Action', 'General', 'Platformer', 'Platformer..."
4,.detuned,"'Action', 'Miscellaneous', 'Rhythm', 'Music', ..."


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 기존 게임 데이터프레임과 유저 게임 데이터프레임 통합
combined_df = pd.concat([db_df, user_games], ignore_index=True)

# TF-IDF 벡터화 객체 생성
vectorizer = TfidfVectorizer()

# 게임 장르 데이터를 TF-IDF 벡터로 변환
genre_vectors = vectorizer.fit_transform(combined_df['genre'])

# 추천을 위한 유사도 측정
similarities = cosine_similarity(genre_vectors[-2:], genre_vectors[:-2])
top_similar_indices = similarities.argsort()[0][::-1][:5] #5개 추천

# 추천 게임 목록 생성
recommendations = combined_df.loc[top_similar_indices, 'name'].tolist()

# 결과 출력
print(f"\n추천하는 게임:")
for i, game in enumerate(recommendations):
    print(f"{i+1}. {game}")


추천하는 게임:
1. Call of Duty: Infinite Warfare - Sabotage
2. Sniper Elite 4
3. Day of Infamy
4. SUPERHOT VR
5. Arktika.1
